In [1]:
!pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 7.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 6.1 MB/s eta 0:00:0000:01


--- Iniciando conexión a InfluxDB ---
Verificando estado de salud de InfluxDB en http://influxdb2:8086...
[INFO] ¡Conexión exitosa!
[INFO]  Versión del servidor: v2.7.12
--- Conexión cerrada ---


In [11]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0="

print("--- Iniciando conexión a InfluxDB ---")

client = None
try:
    # 1. Inicializar el cliente
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="dev_pruebas"
    )

    # 2. Verificar la conexión con el servidor (usando ping() en lugar de health())
    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    
    # Usar ping() que es el método recomendado
    version = client.ping()
    
    if version:
        print("[INFO] ¡Conexión exitosa!")
        print(f"[INFO] Versión del servidor: {version}")
    else:
        print("[ERROR] No se pudo conectar al servidor - ping retornó False")

except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")
    
except Exception as e:
    print(f"[ERROR] Error inesperado: {e}")
    

--- Iniciando conexión a InfluxDB ---
Verificando estado de salud de InfluxDB en http://influxdb2:8086...
[INFO] ¡Conexión exitosa!
[INFO] Versión del servidor: True


In [13]:
write_api = client.write_api(write_options=SYNCHRONOUS)
import datetime
from influxdb_client import Point

# Usar el tiempo actual en formato ISO 8601 (preferido)
tiempo_actual = datetime.datetime.utcnow().isoformat() + "Z"

p = (
    Point("datos_iot")
    .tag("ubicacion", "bodega_sur")
    .tag("sensor_id", "S003")
    .field("humedad_relativa", 75.8)
    .field("nivel_bateria", 95)
    .field("activo", True)
    .time(tiempo_actual)
)  # Paréntesis de cierre corregido
write_api.write(bucket="prueba1", org="docs", record=p)
